In [25]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import networkx as nx
import igraph as ig
from tqdm import tqdm
import numpy as np

列表格式和铺平格式相互转化

In [93]:
def flatten_active(df): 
    df['app_len'] = df['appid'].apply(lambda x: len(x))
    u = []
    a = []
    for i in tqdm(range(len(df['appid'].values))):
        u += [df['uid'].values[i]]*df['app_len'].values[i]
        a += list(df['appid'].values[i])
        
    new_df = pd.DataFrame()
    new_df['uid'] = u
    new_df['appid'] = a
        
    return new_df

In [94]:
df1 =pd.DataFrame([[101,[1,2,5]],[102,[2,3,5]],[103,[2,4,1]],[104,[2,3]]],columns=['uid','appid'])

In [102]:
df1

,uid,appid,app_len
0,101,"[1, 2, 5]",3
1,102,"[2, 3, 5]",3
2,103,"[2, 4, 1]",3
3,104,"[2, 3]",2


In [99]:
df2 = flatten_active(df1)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]


In [103]:
df2

,uid,appid
0,101,1
1,101,2
2,101,5
3,102,2
4,102,3
5,102,5
6,103,2
7,103,4
8,103,1
9,104,2


In [101]:
df2[['uid','appid']].groupby(['uid'])['appid'].apply(lambda x:list(x)).reset_index()

,uid,appid
0,101,"[1, 2, 5]"
1,102,"[2, 3, 5]"
2,103,"[2, 4, 1]"
3,104,"[2, 3]"


从图统计量的角度和prone的角度分别挖掘特征

In [16]:
# Graph Feature
import scipy.sparse
from scipy import linalg
from scipy.special import iv
import scipy.sparse as sp

from sklearn import preprocessing
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD

import argparse
import time

class ProNE():
    def __init__(self, G, emb_size=128, step=10, theta=0.5, mu=0.2, n_iter=5, random_state=2019):
        self.G = G
        self.emb_size = emb_size
        self.G = self.G.to_undirected()
        self.node_number = self.G.number_of_nodes()
        self.random_state = random_state
        self.step = step
        self.theta = theta
        self.mu = mu
        self.n_iter = n_iter
        
        mat = scipy.sparse.lil_matrix((self.node_number, self.node_number))

        for e in tqdm(self.G.edges()):
            if e[0] != e[1]:
                mat[int(e[0]), int(e[1])] = 1
                mat[int(e[1]), int(e[0])] = 1
        self.mat = scipy.sparse.csr_matrix(mat)
        print(mat.shape)

    def get_embedding_rand(self, matrix):
        # Sparse randomized tSVD for fast embedding
        t1 = time.time()
        l = matrix.shape[0]
        smat = scipy.sparse.csc_matrix(matrix)  # convert to sparse CSC format
        print('svd sparse', smat.data.shape[0] * 1.0 / l ** 2)
        U, Sigma, VT = randomized_svd(smat, n_components=self.emb_size, n_iter=self.n_iter, random_state=self.random_state)
        U = U * np.sqrt(Sigma)
        U = preprocessing.normalize(U, "l2")
        print('sparsesvd time', time.time() - t1)
        return U

    def get_embedding_dense(self, matrix, emb_size):
        # get dense embedding via SVD
        t1 = time.time()
        U, s, Vh = linalg.svd(matrix, full_matrices=False, check_finite=False, overwrite_a=True)
        U = np.array(U)
        U = U[:, :emb_size]
        s = s[:emb_size]
        s = np.sqrt(s)
        U = U * s
        U = preprocessing.normalize(U, "l2")
        print('densesvd time', time.time() - t1)
        return U

    def fit(self, tran, mask):
        # Network Embedding as Sparse Matrix Factorization
        t1 = time.time()
        l1 = 0.75
        C1 = preprocessing.normalize(tran, "l1")
        neg = np.array(C1.sum(axis=0))[0] ** l1

        neg = neg / neg.sum()

        neg = scipy.sparse.diags(neg, format="csr")
        neg = mask.dot(neg)
        print("neg", time.time() - t1)

        C1.data[C1.data <= 0] = 1
        neg.data[neg.data <= 0] = 1

        C1.data = np.log(C1.data)
        neg.data = np.log(neg.data)

        C1 -= neg
        F = C1
        features_matrix = self.get_embedding_rand(F)
        return features_matrix

    def chebyshev_gaussian(self, A, a, order=10, mu=0.5, s=0.5):
        # NE Enhancement via Spectral Propagation
        print('Chebyshev Series -----------------')
        t1 = time.time()

        if order == 1:
            return a

        A = sp.eye(self.node_number) + A
        DA = preprocessing.normalize(A, norm='l1')
        L = sp.eye(self.node_number) - DA

        M = L - mu * sp.eye(self.node_number)

        Lx0 = a
        Lx1 = M.dot(a)
        Lx1 = 0.5 * M.dot(Lx1) - a

        conv = iv(0, s) * Lx0
        conv -= 2 * iv(1, s) * Lx1
        for i in range(2, order):
            Lx2 = M.dot(Lx1)
            Lx2 = (M.dot(Lx2) - 2 * Lx1) - Lx0
            #         Lx2 = 2*L.dot(Lx1) - Lx0
            if i % 2 == 0:
                conv += 2 * iv(i, s) * Lx2
            else:
                conv -= 2 * iv(i, s) * Lx2
            Lx0 = Lx1
            Lx1 = Lx2
            del Lx2
            print('Bessell time', i, time.time() - t1)
        mm = A.dot(a - conv)
        self.embeddings = self.get_embedding_dense(mm, self.emb_size)
        return self.embeddings
    
    def transform(self):
        if self.embeddings is None:
            print("Embedding is not train")
            return {}
        self.embeddings = pd.DataFrame(self.embeddings)
        self.embeddings.columns = ['ProNE_Emb_{}'.format(i) for i in range(len(self.embeddings.columns))]
        self.embeddings = self.embeddings.reset_index().rename(columns={'index' : 'nodes'}).sort_values(by=['nodes'],ascending=True).reset_index(drop=True)

        return self.embeddings

避免uid和appid重复进行重新编码

In [104]:
uid_lbl,appid_lbl = LabelEncoder(),LabelEncoder()
df2['new_uid'] = uid_lbl.fit_transform(df2['uid'])
df2['new_appid'] = appid_lbl.fit_transform(df2['appid'])
df2['new_appid'] += df2['new_uid'].max() + 1

In [105]:
df2

,uid,appid,new_uid,new_appid
0,101,1,0,4
1,101,2,0,5
2,101,5,0,8
3,102,2,1,5
4,102,3,1,6
5,102,5,1,8
6,103,2,2,5
7,103,4,2,7
8,103,1,2,4
9,104,2,3,5


In [68]:
def get_graph_embedding(df,prefix):
    
    uid_lbl,appid_lbl = LabelEncoder(),LabelEncoder()
    df['new_uid'] = uid_lbl.fit_transform(df['uid'])
    df['new_appid'] = appid_lbl.fit_transform(df['appid'])
    df['new_appid'] += df['new_uid'].max() + 1
    
    print("Encoder Finished...")
    
    G = ig.Graph()
    G.add_vertices(df['new_appid'].max()+1)
    G.add_edges(df[['new_uid','new_appid']].values)
    print("Build Graph Finished...")
    evcent = G.evcent() # 计算图中节点的向量中心性
    shell_index = G.shell_index() # 计算图中节点度至少为K的最大子图
    degree = G.degree() # 总度数
    pagerank = G.pagerank() # pagerank
    # 以下4个计算的很慢，效果还不错，可以注释掉，观察evcent的效果
    closeness = G.closeness() # 计算节点与网络中其他所有节点的距离的平均值   
    betweenness = G.betweenness() # 计算节点的介值
    constraint = G.constraint()
    eccentricity = G.eccentricity() # 计算给定节点到图中其他节点的最短距离的最大值。
    
    G_stat = pd.DataFrame()
    G_stat['evcent'] = evcent
    G_stat['shell_index'] = shell_index
    G_stat['degree'] = degree
    G_stat['pagerank'] = pagerank
    print("PR Finished...")
    G_stat['closeness'] = closeness
    G_stat['betweenness'] = betweenness
    G_stat['constraint'] = constraint
    G_stat['eccentricity'] = eccentricity
    G_stat = G_stat.reset_index()
    G_stat = G_stat[G_stat['index'].isin(df['new_uid'])]
    G_stat['index'] = uid_lbl.inverse_transform(G_stat['index'])
    G_stat.rename(columns={'index' : 'uid'},inplace=True)
    
    print("Graph Stat Finished...")
    G_stat.to_pickle("Graph_Stat_{}.pickle".format(prefix))
    
    del G
    
    import gc
    gc.collect()
    
    G = nx.Graph()
    G.add_edges_from(df[['new_uid','new_appid']].values)
    model = ProNE(G,emb_size=32,n_iter=6,step=12)
    features_matrix = model.fit(model.mat, model.mat)
    model.chebyshev_gaussian(model.mat, features_matrix, model.step, model.mu, model.theta)
    emb = model.transform()
    fea = emb[emb['nodes'].isin(df['new_uid'])]
    fea['nodes'] = uid_lbl.inverse_transform(fea['nodes'])
    fea.rename(columns={'nodes' : 'uid'},inplace=True)
    del G
    gc.collect()
    print("Embedding Finished...")
    fea.to_pickle("Graph_Bi_{}.pickle".format(prefix))
    
    return fea,G_stat

In [106]:
#test为生成文件的区别标签，add_prefix为特征的区别标签。
fea1,stat1 = get_graph_embedding(df2,'test')

Encoder Finished...
Build Graph Finished...
PR Finished...
Graph Stat Finished...


100%|███████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 11024.45it/s]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15620\2273061183.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fea['nodes'] = uid_lbl.inverse_transform(fea['nodes'])
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15620\2273061183.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fea.rename(columns={'nodes' : 'uid'},inplace=True)


(9, 9)
neg 0.0009970664978027344
svd sparse 0.2716049382716049
sparsesvd time 0.0019948482513427734
Chebyshev Series -----------------
Bessell time 2 0.0009970664978027344
Bessell time 3 0.0009970664978027344
Bessell time 4 0.0009970664978027344
Bessell time 5 0.0009970664978027344
Bessell time 6 0.0009970664978027344
Bessell time 7 0.0009970664978027344
Bessell time 8 0.0009970664978027344
Bessell time 9 0.001994609832763672
Bessell time 10 0.001994609832763672
Bessell time 11 0.001994609832763672
densesvd time 0.0
Embedding Finished...


In [107]:
stat1

,uid,evcent,shell_index,degree,pagerank,closeness,betweenness,constraint,eccentricity
0,101,0.767539,2,3,0.131348,0.533333,5.433333,0.333333,3.0
1,102,0.751072,2,3,0.131051,0.533333,5.600000,0.333333,3.0
2,103,0.649428,2,3,0.140726,0.533333,8.733333,0.333333,3.0
3,104,0.544711,2,2,0.092371,0.470588,2.233333,0.500000,3.0


In [108]:
fea1

,uid,ProNE_Emb_0,ProNE_Emb_1,ProNE_Emb_2,ProNE_Emb_3,ProNE_Emb_4,ProNE_Emb_5,ProNE_Emb_6,ProNE_Emb_7,ProNE_Emb_8
0,101,-0.719094,0.306798,-0.519837,-0.197759,0.201183,0.186374,0.001282,-0.065027,-3.167950e-09
1,102,-0.740016,-0.452264,-0.152969,0.318751,0.211357,-0.275946,0.039074,-0.022073,4.503036e-10
2,103,-0.552247,0.667918,0.404282,0.122372,0.155979,0.003941,-0.206554,0.058999,-1.419043e-10
3,104,-0.597442,-0.487065,0.469144,-0.295471,0.173153,0.129794,0.217487,0.065586,1.849583e-09
